# Morris sampling sensitivity analysis for Forrest local SDGs systems model

In [1]:
#set working directory locations
import sys
sys.path.append(r'C:\temp')
sys.path.append(r'C:\Users\szeteyka\OneDrive - Deakin University\PhD\Models\Moallemi_et_al_SDG_SSP_Assessment-main')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ema_workbench import load_results, ema_logging
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import morris
import os
from datetime import datetime

C:\Users\szeteyka\Anaconda3\lib\site-packages\ema_workbench\em_framework\evaluators.py:21: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn(
C:\Users\szeteyka\Anaconda3\lib\site-packages\ema_workbench\em_framework\optimization.py:48: ImportWarning: platypus based optimization not available
  warnings.warn("platypus based optimization not available", ImportWarning)


## Load model, uncertainities, outcomes; Run experiments

### before running this code, either rename the save file or delete the previous version

In [3]:
def generate_experiments(n_scenarios):
    
    from ema_workbench.connectors.vensim import VensimModel
    #set working directory
    directory = 'C:/temp/wd/'
    #set model file location
    vensimModel = VensimModel("LocalSDGsmodel", wd=directory, model_file=r'C:/temp/model/LocalSDGsmodel.vpmx')
    
    from ema_workbench import (TimeSeriesOutcome, 
                                   perform_experiments,
                                   RealParameter, 
                                   CategoricalParameter,
                                   ema_logging, 
                                   save_results,
                                  load_results)
    #set working directory
    directory = 'C:/temp/model/'

    #define file and sheet for uncertainties
    df_unc = pd.read_excel(directory+'SensitivityVariables.xlsx', sheet_name='uncertainties')
    
    #define column headers for lower and upper bounds of uncertainties
    vensimModel.uncertainties = [RealParameter(row['Variable'], row['Lower'], row['Upper']) for index, row in df_unc.iterrows()]

    #define file and sheet for outcomes
    df_out = pd.read_excel(directory+'SensitivityVariables.xlsx', sheet_name='outcomes')
    
    #define column headers for outcomes
    vensimModel.outcomes = [TimeSeriesOutcome(out) for out in df_out['Outcome']]

    from ema_workbench import MultiprocessingEvaluator
    from ema_workbench.em_framework.evaluators import (MC, LHS, FAST, FF, PFF, SOBOL, MORRIS)


    try: 
        with MultiprocessingEvaluator(vensimModel, n_processes=3) as evaluator:
            results = evaluator.perform_experiments(scenarios=n_scenarios, uncertainty_sampling=MORRIS)
    except (BrokenPipeError, IOError):
        pass

    #define results location and filename
    fn = 'C:/temp/results/Forrest_exps_sc{}.tar.gz'.format(n_scenarios)

    # If scenario results file exists, move to backup (replacing the backup if it exists)
    
    if os.path.isfile(fn):
        os.replace(fn, fn + '.bak')
    
    save_results(results, fn)
    
    experiments, outcomes = results
    
    return vensimModel.uncertainties, experiments, outcomes, results

In [4]:
# If Morris index .csv files already exist in the save directory, you MUST delete them as this code does not overwrite them

def make_morris_df(scores, problem, outcome_var, sc, t):
    # print("make_morris_df {}".format(outcome_var))
    scores_filtered = {k:scores[k] for k in ['mu_star','mu_star_conf','mu','sigma']}
    Si_df = pd.DataFrame(scores_filtered, index=problem['names'])
    
    #define location to save results
    sa_dir = 'C:/temp/results/'
    Si_df.to_csv(sa_dir+"MorrisIndices_{}_sc{}_t{}_test.csv".format(outcome_var, sc, t))
    
    Si_df.sort_values(by=['mu_star'], ascending=False, inplace=True)
    Si_df = Si_df.head(20)
    
    Si_df = Si_df.iloc[::-1]

    indices = Si_df[['mu_star','mu']]
    errors = Si_df[['mu_star_conf','sigma']]
    return indices, errors

In [5]:
def plot_scores(inds, errs, outcome_var, sc, t):

    sns.set_style('white')
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(3, 6)) 
    ind = inds[outcome_var].iloc[:,0]
    err = errs[outcome_var].iloc[:,0]
    ind.plot.barh(xerr=err.values.T,ax=ax, color = ['#62C890'], width=.9)
    ax.set_ylabel('')
    ax.legend().set_visible(False)
    ax.set_xlabel('mu_star index', fontsize=14)

    ylabels = ax.get_yticklabels()
    ax.set_yticklabels(ylabels, fontsize=10)

    # datetime object containing current date and time
    now = datetime.now()
 
    #format as YYmmdd-HHMM
    dt_string = now.strftime("%Y%m%d-%H%M")

    plt.suptitle('{} in {}'.format(outcome_var, t), y=0.94, fontsize=12)
    plt.rcParams["figure.figsize"] = [7.08,7.3]
    #define location and filename for morris sensitivity plots
    plt.savefig('{}{}-Morris_ranking_{}_sc{}_{}.png'.format(r'C:/temp/figs/', dt_string, outcome_var, sc, t), 
                dpi=600,  bbox_inches='tight')
    return fig

In [6]:
if __name__ == '__main__':
    
    ema_logging.log_to_stderr(ema_logging.INFO)

    targetyear_ind = {2030: 30}
    import  time
    start = time.time()
    for n_scenarios in [2000]:
        #print("Starting {} scenarios".format(n_scenarios))
        uncertainties, experiments, outcomes, results = generate_experiments(n_scenarios)
        sc = n_scenarios
        outcome_vars = list(outcomes.keys())[1:]
        problem = get_SALib_problem(uncertainties)
        for t in list(targetyear_ind.keys()):
            #print("Year {}".format(t))
            inds ={}
            errs = {}
            for i, outcome_var in enumerate(outcome_vars):
                # print("i {} outcome_var {}".format(i,outcome_var))
                X = experiments.iloc[:, :-3].values
                Y = outcomes[outcome_var][:,targetyear_ind[t]]
                scores = morris.analyze(problem, X, Y, print_to_console=False)
                inds[outcome_var], errs[outcome_var] = make_morris_df(scores, problem, outcome_var, sc, t)
            for out, outcome_var in enumerate(outcome_vars):
                plot_scores(inds, errs, outcome_var, sc, t)
                plt.close()
    end = time.time()
    print("took {} seconds".format(end-start))

[MainProcess/INFO] using 64 bit vensim
C:\Users\szeteyka\Anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:27: ImportWarning: netlogo connector not available
  warnings.warn("netlogo connector not available", ImportWarning)
[MainProcess/INFO] pool started
[MainProcess/INFO] performing 58000 scenarios * 1 policies * 1 model(s) = 58000 experiments
[MainProcess/INFO] 5800 cases completed
[MainProcess/INFO] 11600 cases completed
[MainProcess/INFO] 17400 cases completed
[MainProcess/INFO] 23200 cases completed
[MainProcess/INFO] 29000 cases completed
[MainProcess/INFO] 34800 cases completed
[MainProcess/INFO] 40600 cases completed
[MainProcess/INFO] 46400 cases completed
[MainProcess/INFO] 52200 cases completed
[MainProcess/INFO] 58000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] results saved successfully to C:\temp\results\Forrest_exps_sc2000.tar.gz


took 16025.942645311356 seconds


In [28]:
    # Run just the plots (debugging)
    
    start = time.time()
    for n_scenarios in [2000]:
        #print("Starting {} scenarios".format(n_scenarios))
        #uncertainties, experiments, outcomes, results = generate_experiments(n_scenarios)
        sc = n_scenarios
        outcome_vars = list(outcomes.keys())[1:]
        problem = get_SALib_problem(uncertainties)
        for t in list(targetyear_ind.keys()):
            #print("Year {}".format(t))
            inds ={}
            errs = {}
            for i, outcome_var in enumerate(outcome_vars):
                # print("i {} outcome_var {}".format(i,outcome_var))
                X = experiments.iloc[:, :-3].values
                Y = outcomes[outcome_var][:,targetyear_ind[t]]
                scores = morris.analyze(problem, X, Y, print_to_console=False)
                inds[outcome_var], errs[outcome_var] = make_morris_df(scores, problem, outcome_var, sc, t)
            for out, outcome_var in enumerate(outcome_vars):
                plot_scores(inds, errs, outcome_var, sc, t)
                plt.close()
    end = time.time()
    print("took {} seconds".format(end-start))

took 9.306471109390259 seconds


In [7]:
#save_results(results, fn)
    
experiments, outcomes = results
    
#return vensimModel.uncertainties, experiments, outcomes, results

In [8]:
#load results from previously defined save location

results = load_results('C:/temp/results/Forrest_exps_sc2000.tar.gz')
experiments, outcomes = results
dict_outcomes = outcomes

[MainProcess/INFO] results loaded succesfully from C:\temp\results\Forrest_exps_sc2000.tar.gz


In [9]:
#generate dataframe of outcome results

outcomes_df = pd.DataFrame(columns=list(dict_outcomes.keys()))
for key in set(list(dict_outcomes.keys())): 
    outcomes_df[key] = dict_outcomes[key][:, 0:].ravel() # Specify from when to plot the data 
outcomes_dict = outcomes_df

outcomes_df

,TIME,total population,Housing Land,housing demand,Total Other Productivity,Species Richness,inequality indicator,Safer Healthier People,Internet Service Demand,bus frequency required for travel equity
0,2000.00,171.00000,57.300000,85.500000,1175000.0,3246.0000,0.262923,171.00000,141.00000,516.00000
1,2000.25,171.06958,77.780495,85.534790,1184413.1,3241.4550,0.261234,170.86040,141.02034,513.07120
2,2000.50,171.11960,92.663230,85.606445,1193906.5,3237.0837,0.261567,170.72153,141.04195,509.97452
3,2000.75,171.15282,105.002350,85.716340,1203416.1,3232.8680,0.259286,170.58336,141.06482,506.73654
4,2001.00,171.17351,115.767296,85.866610,1212941.9,3228.8013,0.259150,170.44589,141.08884,503.40234
...,...,...,...,...,...,...,...,...,...,...
11657995,2049.00,285.08597,561.448060,183.933100,4491062.5,4802.2803,0.245940,127.14754,149.44664,607.25390
11657996,2049.25,286.11288,562.135560,184.446550,4510983.5,4817.4710,0.245954,126.98642,149.50397,609.12805
11657997,2049.50,287.14398,562.817000,184.962110,4530941.5,4832.6973,0.245968,126.82639,149.56145,611.01580
11657998,2049.75,288.17932,563.492550,185.479780,4550855.5,4847.9580,0.245983,126.66744,149.61908,612.91670


In [10]:
#print outcomes to csv file

# datetime object containing current date and time
now = datetime.now()
 
#format as YYmmdd-HHMM
dt_string = now.strftime("%Y%m%d-%H%M")

#define save file location for results and export to csv
outcomes_df.to_csv('C:/temp/results/{}_morris_outcomes.csv'.format(dt_string))